In [1]:
import pandas as pd
# Import CSV files
monas_data = pd.read_csv("../data/raw/norcoast_overview.csv")
seg_results = pd.read_csv('../data/raw/seg_results.csv')

In [38]:
def make_hospital_code(hospital_name):
    if hospital_name == 'St. Olavs Hospital':
        return '400'
    elif hospital_name == 'Haukeland Universitetssykehus':
        return '300'
    elif hospital_name == 'Vestre Viken HF, Sykehuset Asker og Bær':
        return '221'
    elif hospital_name == 'Oslo Universitetssykehus, Ullevål':
        return '100'
    elif hospital_name == 'Ålesund sykehus':
        return '410'

def fill_zeros(pno):
    pno_as_string = str(pno)
    filled_with_zeros = pno_as_string.zfill(3)
    return filled_with_zeros

def check_if_images(pno, list_over_patients):
    if pno in list_over_patients:
        return 1
    else:
        return 0

def make_hospital_name(pid):
    first_three_numbers = pid[:3]
    if first_three_numbers == '400':
        return 'St. Olavs'
    if first_three_numbers == '300':
        return 'Haukeland'
    if first_three_numbers == '221':
        return 'Bærum'
    if first_three_numbers == '100':
        return 'Ullevål'
    if first_three_numbers == '410':
        return 'Ålesund'

In [39]:
# monas_data.head()
# Antall pasienter i excel fil som Mona har vist
#
number_of_unique_patients_in_excel_fil = monas_data['PatientName'].nunique()
# Antall linjer med modalitet anført som CT
ct_data = monas_data.loc[monas_data['Modality'] == 'CT',['PatientName','SeriesDescription','StudyDescription']]
antall_pasienter_med_ct_i_excel_file = ct_data['PatientName'].nunique()
# Liste over pasienter med bilder i excel filen
list_over_patients_with_images = [str(x) for x in list(monas_data['PatientName'].unique())]
# Liste over pasienter med CT i excel filen
ct_list = [str(x) for x in list(ct_data['PatientName'].unique())]
seg_results['Hosp_x'].value_counts()

St. Olavs Hospital                         400
Haukeland Universitetssykehus              142
Vestre Viken HF, Sykehuset Asker og Bær    141
Oslo Universitetssykehus, Ullevål           99
Ålesund sykehus                             33
Name: Hosp_x, dtype: int64

In [40]:
cts_and_hospital = pd.DataFrame({'nc_id': ct_list, 'hosp': [make_hospital_name(x) for x in ct_list]})
cts_and_hospital['hosp'].value_counts()

St. Olavs    231
Bærum         10
Name: hosp, dtype: int64

In [41]:
seg_results['hosp_code'] = seg_results['Hosp_x'].apply(make_hospital_code)
seg_results['patient_code'] = seg_results['pNo'].apply(fill_zeros)
seg_results['NorCOAST_ID'] = seg_results['hosp_code'] + seg_results['patient_code']
seg_results['clinical_images'] = seg_results['NorCOAST_ID'].apply(check_if_images, list_over_patients=list_over_patients_with_images)
seg_results['ct_images'] = seg_results['NorCOAST_ID'].apply(check_if_images, list_over_patients=ct_list)

In [42]:
seg_results.loc[seg_results['clinical_images'] == False, 'Hosp_x'].value_counts()

Haukeland Universitetssykehus              96
St. Olavs Hospital                         85
Vestre Viken HF, Sykehuset Asker og Bær    54
Oslo Universitetssykehus, Ullevål          42
Ålesund sykehus                            25
Name: Hosp_x, dtype: int64

In [43]:
seg_results.loc[seg_results['clinical_images'] == True, 'Hosp_x'].value_counts()

St. Olavs Hospital                         315
Vestre Viken HF, Sykehuset Asker og Bær     87
Oslo Universitetssykehus, Ullevål           57
Haukeland Universitetssykehus               46
Ålesund sykehus                              8
Name: Hosp_x, dtype: int64

In [44]:
seg_results.to_csv('../data/raw/nc_id.csv')

In [45]:
seg_results.loc[seg_results['ct_images'] == True, 'Hosp_x'].value_counts()

St. Olavs Hospital                         231
Vestre Viken HF, Sykehuset Asker og Bær      9
Name: Hosp_x, dtype: int64

241

In [47]:
%connect_info

{
  "shell_port": 58153,
  "iopub_port": 57715,
  "stdin_port": 45059,
  "control_port": 40069,
  "hb_port": 36039,
  "ip": "127.0.0.1",
  "key": "c084e3fd-be49e6d364418b5c8d5451db",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-681f9d96-9e84-4f92-8066-cef171b50394.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [12]:
%qtconsole